In [1]:
import os
current_directory = os.getcwd()
folder_name = os.path.basename(current_directory)
number = folder_name

In [2]:
import platform
platform_name = platform.system()

In [3]:
CONFIG = {
    'test_path': 'test.csv',
    'train_path': 'train.csv',

    'DEVICE' : 'cuda',
    'SEED' : 42,
}
if platform_name == 'Windows':
    CONFIG['data_main'] = 'C:/Users/Николай/PycharmProjects/CIBMTR/D.Data/main/'
    CONFIG['model_path'] = f"{number}_model.pkl"
else:
    CONFIG['data_main'] = '/kaggle/input/equity-post-HCT-survival-predictions/'
    CONFIG['model_path'] = "/kaggle/input/2.1.6_model.pkl/other/default/1/2.1.6_model.pkl"

In [4]:

import pandas as pd
import joblib 

In [5]:
train = pd.read_csv(f"{CONFIG['data_main']}{CONFIG['train_path']}")

num_columns = [col for col in train.columns if pd.api.types.is_numeric_dtype(train[col])]
cat_columns = [col for col in train.columns if col not in num_columns]

In [6]:
test = pd.read_csv(f"{CONFIG['data_main']}test.csv")
for col in test.columns: 
    if col in cat_columns:
        test[col] = test[col].astype(str)
        test[col] = test[col].fillna('-1')
    elif col in num_columns:
        test[col] = test[col].fillna(test[col].mode()[0])

In [7]:
final_cat = joblib.load(CONFIG['model_path'])

In [8]:
final_cat.get_params()

{'iterations': 17455,
 'learning_rate': 0.004493946464004274,
 'depth': 11,
 'l2_leaf_reg': 4.995727127215648,
 'model_size_reg': 0,
 'rsm': 1,
 'loss_function': 'RMSE',
 'border_count': 952,
 'feature_border_type': 'GreedyLogSum',
 'nan_mode': 'Forbidden',
 'leaf_estimation_iterations': 1,
 'leaf_estimation_method': 'Newton',
 'random_seed': 42,
 'use_best_model': False,
 'logging_level': 'Silent',
 'max_ctr_complexity': 15,
 'one_hot_max_size': 50,
 'random_strength': 0,
 'eval_metric': 'RMSE',
 'bagging_temperature': 0.9991831740725914,
 'allow_writing_files': False,
 'task_type': 'GPU',
 'bootstrap_type': 'Bayesian',
 'early_stopping_rounds': 250,
 'cat_features': ['dri_score',
  'psych_disturb',
  'cyto_score',
  'diabetes',
  'tbi_status',
  'arrhythmia',
  'graft_type',
  'vent_hist',
  'renal_issue',
  'pulm_severe',
  'prim_disease_hct',
  'cmv_status',
  'tce_imm_match',
  'rituximab',
  'prod_type',
  'cyto_score_detail',
  'conditioning_intensity',
  'ethnicity',
  'obesity

In [9]:
preds_test = final_cat.predict(test.drop(columns=['ID']))

In [10]:
y_pred = test[['ID']].copy(deep=True)
y_pred["prediction"] = preds_test
y_pred.to_csv('submission.csv', index=False)

In [11]:
test

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,karnofsky_score,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10
0,28800,N/A - non-malignant indication,No,nan,No,2.0,8.0,No TBI,No,6.0,...,90.0,No,nan,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0
1,28801,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,...,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0
2,28802,N/A - non-malignant indication,No,nan,No,2.0,8.0,No TBI,No,6.0,...,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0
